In [ ]:
# default_exp export_steps

# Export Step for a Flow

In [1]:
# exportn

In [60]:
from nbdev.export import (is_export, read_nb, find_default_export, 
                          _is_external_export, _re_internal_export, _mk_flag_re,
                         check_re, _re_mod_export, _re_blank_export)
from pathlib import Path

In [138]:
nb_path = Path('/home/jovyan/git/sciflow/examples/top2vec.ipynb')

In [139]:
nb = read_nb(nb_path)
default = find_default_export(nb['cells'])
exports = [is_export(c, default) for c in nb['cells']]

In [140]:
[c for c in nb['cells']][0]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {},
 'outputs': [],
 'source': '# exportn_step:prepproc\n\ndef prepproc():\n    print(1)'}

In [147]:
check_re([c for c in nb['cells']][0], 'exportn_[a-zA-Z0-9:]+')

<re.Match object; span=(2, 23), match='exportn_step:prepproc'>

In [149]:
_re_named_export = _mk_flag_re("exportn_[a-zA-Z0-9:]+", 0,
    "Matches any line with a named #exportn without any module name")

In [152]:
def extract_named_export(cell):
    name = None
    tst = check_re(cell, _re_named_export)
    if tst:
        pat = re.compile(rf"""exportn_[a-zA-Z0-9:]+""", re.MULTILINE | re.VERBOSE)
        name = pat.search(tst.string).group(0).split('_')[1]
    return name

# Test Notebook2Script & add has named export check

In [153]:
[extract_named_export(c) for c in nb['cells']]

['step:prepproc',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'step:preprocess',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [121]:
import re

In [132]:
pat.search(tst.string).group(0).split(':')

['exportn', 'preprocess']

In [64]:
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module to export it if provided"
    tst = check_re(cell, _re_blank_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_mod_export)
    if tst: return os.path.sep.join(tst.groups()[0].split('.')), _is_external_export(tst)
    else: return None

In [18]:
cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]

In [21]:
i,c,(e,a) = cells[0]

In [22]:
i

3

In [24]:
e

'top2vec'

In [25]:
a

True

In [ ]:
def _notebook2script(fname, modules, silent=False, to_dict=None, bare=False):
    "Finds cells starting with `#export` and puts them into a module created by `create_mod_files`"
    bare = str(Config().get('bare', bare)) == 'True'
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    sep = '\n'* (int(Config().get('cell_spacing', '1'))+1)
    fname = Path(fname)
    nb = read_nb(fname)
    default = find_default_export(nb['cells'])
    if default is not None:
        default = os.path.sep.join(default.split('.'))
    mod = get_nbdev_module()
    exports = [is_export(c, default) for c in nb['cells']]
    cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]
    for i,c,(e,a) in cells:
        if e not in modules: print(f'Warning: Exporting to "{e}.py" but this module is not part of this build')
        fname_out = Config().path("lib_path")/f'{e}.py'
        if bare: orig = "\n"
        else: 
            orig = (f'# {"" if a else "Internal "}C' if e==default else f'# Comes from {fname.name}, c') + 'ell\n'
        flag_lines,code_lines = split_flags_and_code(c)
        code_lines = _deal_import(code_lines, fname_out)
        code = sep + orig + '\n'.join(code_lines)
        names = export_names(code)
        flags = '\n'.join(flag_lines)
        extra,code = extra_add(flags, code)
        code = _from_future_import(fname_out, flags, code, to_dict)
        if a:
            if to_dict is None: _add2all(fname_out, [f"'{f}'" for f in names if '.' not in f and len(f) > 0] + extra)
        mod.index.update({f: fname.name for f in names})
        code = re.sub(r' +$', '', code, flags=re.MULTILINE)
        if code != sep + orig[:-1]:
            if to_dict is not None: to_dict[e].append((i, fname, code))
            else:
                with open(fname_out, 'a', encoding='utf8') as f: f.write(code)
        if f'{e}.py' not in mod.modules: mod.modules.append(f'{e}.py')
    save_nbdev_module(mod)

    if not silent: print(f"Converted {fname.name}.")
    return to_dict